# Menyetel Hyperparameter

Ada banyak algoritme pembelajaran mesin yang memerlukan *hyperparameter* (nilai parameter yang memengaruhi pelatihan, tetapi tidak dapat ditentukan dari data pelatihan itu sendiri). Misalnya, saat melatih model regresi logistik, Anda dapat menggunakan hyperparameter *tingkat regularisasi* untuk melawan bias dalam model; atau saat melatih jaringan saraf konvolusional, Anda dapat menggunakan hyperparameter seperti *tingkat pembelajaran* dan *ukuran batch* untuk mengontrol bagaimana berat disesuaikan dan berapa banyak item data yang masing-masing diproses dalam mini-batch. Pilihan nilai hyperparameter dapat secara signifikan mempengaruhi performa model yang dilatih, atau waktu yang dibutuhkan untuk melatihnya; dan seringkali Anda perlu mencoba beberapa kombinasi untuk menemukan solusi optimal.

Dalam hal ini, Anda akan melatih model klasifikasi dengan dua hyperparameter, tetapi prinsipnya berlaku untuk semua jenis model yang dapat Anda latih dengan Azure Machine Learning.

## Menghubungkan ke ruang kerja Anda

Untuk memulai, hubungkan ke ruang kerja Anda.

> **Catatan**: Jika Anda belum membuat sesi yang terautentikasi dengan langganan Azure, Anda akan diminta untuk mengautentikasi dengan mengklik tautan, memasukkan kode autentikasi, dan masuk ke Azure.

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## Mempersiapkan data

Di lab ini, Anda akan menggunakan himpunan data yang berisi detail pasien diabetes. Jalankan sel di bawah ini untuk membuat himpunan data ini (jika sudah ada, versi yang ada akan digunakan)

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

## Menyiapkan skrip pelatihan

Sekarang mari membuat folder untuk skrip pelatihan yang akan Anda gunakan untuk melatih model.

In [ ]:
import os

experiment_folder = 'diabetes_training-hyperdrive'
os.makedirs(experiment_folder, exist_ok=True)

print('Folder ready.')

Sekarang buat skrip Python untuk melatih model. Dalam contoh ini, Anda akan menggunakan algoritme *Peningkatan Gradien* untuk melatih model klasifikasi. Skrip harus mencakup:

- Argumen untuk setiap hyperparameter yang ingin Anda optimalkan (dalam hal ini, kecepatan pembelajaran dan jumlah estimator untuk algoritma Gradient Boosting)
- Kode untuk mencatat metrik performa yang ingin Anda optimalkan (dalam hal ini, Anda akan mencatat AUC dan akurasi, sehingga Anda dapat memilih untuk mengoptimalkan model untuk keduanya)

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# Import libraries
import argparse, joblib, os
from azureml.core import Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, roc_curve

# Get the experiment run context
run = Run.get_context()

# Get script arguments
parser = argparse.ArgumentParser()

# Input dataset
parser.add_argument("--input-data", type=str, dest='input_data', help='training dataset')

# Hyperparameters
parser.add_argument('--learning_rate', type=float, dest='learning_rate', default=0.1, help='learning rate')
parser.add_argument('--n_estimators', type=int, dest='n_estimators', default=100, help='number of estimators')

# Add arguments to args collection
args = parser.parse_args()

# Log Hyperparameter values
run.log('learning_rate',  np.float(args.learning_rate))
run.log('n_estimators',  np.int(args.n_estimators))

# load the diabetes dataset
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe() # Get the training data from the estimator input

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a Gradient Boosting classification model with the specified hyperparameters
print('Training a classification model')
model = GradientBoostingClassifier(learning_rate=args.learning_rate,
                                   n_estimators=args.n_estimators).fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Save the model in the run outputs
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## Memuat komputasi

Penyetelan hyperparameter melibatkan menjalankan beberapa perulangan pelatihan dengan nilai hyperparameter yang berbeda dan membandingkan metrik performa dari model yang dihasilkan. Untuk melakukan ini secara efisien, kami akan memanfaatkan komputasi cloud sesuai permintaan dan membuat kluster - ini akan memungkinkan beberapa perulangan pelatihan dijalankan secara bersamaan.

Gunakan kode berikut untuk menentukan kluster komputasi Azure Machine Learning (itu akan dibuat jika belum ada).

> **Penting**: Ubah *your-compute-cluster* menjadi nama kluster komputasi Anda dalam kode di bawah ini sebelum menjalankannya! Nama kluster harus berupa nama unik global dengan panjang antara 2 hingga 16 karakter. Karakter yang valid adalah huruf, angka, dan - karakter.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

> **Catatan**: Instans dan kluster komputasi didasarkan pada gambar mesin virtual Azure standar. Untuk latihan ini, gambar *Standard_DS11_v2* direkomendasikan untuk mencapai keseimbangan optimal antara biaya dan performa. Jika langganan Anda memiliki kuota yang tidak menyertakan gambar ini, pilih gambar alternatif; tetapi perlu diingat bahwa gambar yang lebih besar mungkin memerlukan biaya lebih tinggi dan gambar yang lebih kecil mungkin tidak cukup untuk menyelesaikan tugas. Atau, minta administrator Azure Anda untuk memperbesar kuota Anda.

Anda memerlukan lingkungan Python untuk dihosting di komputasi, jadi mari kita definisikan itu sebagai file konfigurasi Conda.

In [ ]:
%%writefile $experiment_folder/hyperdrive_env.yml
name: batch_environment
dependencies:
- python=3.6.2
- scikit-learn
- pandas
- numpy
- pip
- pip:
  - azureml-defaults


## Menjalankan eksperimen penyetelan hyperparameter

Azure Machine Learning mencakup kemampuan penyetelan hyperparameter melalui eksperimen *hyperdrive*. Eksperimen ini meluncurkan beberapa eksekusi turunan, masing-masing dengan kombinasi hyperparameter yang berbeda. Proses yang menghasilkan model terbaik (sebagaimana ditentukan oleh metrik performa target yang dicatat yang ingin Anda optimalkan) dapat diidentifikasi, dan model terlatihnya dipilih untuk pendaftaran dan penyebaran.

> **Catatan**: Dalam contoh ini, kami tidak menentukan kebijakan penghentian dini. Kebijakan seperti itu hanya relevan jika skrip pelatihan melakukan beberapa iterasi pelatihan, mencatat metrik utama untuk setiap perulangan. Pendekatan ini biasanya digunakan saat melatih model jaringan saraf dalam melalui beberapa *epochs*.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.train.hyperdrive import GridParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice
from azureml.widgets import RunDetails

# Create a Python environment for the experiment
hyper_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/hyperdrive_env.yml")

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                # Add non-hyperparameter arguments -in this case, the training dataset
                                arguments = ['--input-data', diabetes_ds.as_named_input('training_data')],
                                environment=hyper_env,
                                compute_target = training_cluster)

# Sample a range of parameter values
params = GridParameterSampling(
    {
        # Hyperdrive will try 6 combinations, adding these as script arguments
        '--learning_rate': choice(0.01, 0.1, 1.0),
        '--n_estimators' : choice(10, 100)
    }
)

# Configure hyperdrive settings
hyperdrive = HyperDriveConfig(run_config=script_config, 
                          hyperparameter_sampling=params, 
                          policy=None, # No early stopping policy
                          primary_metric_name='AUC', # Find the highest AUC metric
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6, # Restict the experiment to 6 iterations
                          max_concurrent_runs=2) # Run up to 2 iterations in parallel

# Run the experiment
experiment = Experiment(workspace=ws, name='mslearn-diabetes-hyperdrive')
run = experiment.submit(config=hyperdrive)

# Show the status in the notebook as the experiment runs
RunDetails(run).show()
run.wait_for_completion()

Anda dapat melihat status percobaan berjalan di widget di atas. Anda juga dapat melihat eksperimen Hyperdrive utama yang dijalankan dan turunannya yang berjalan di [Azure Machine Learning studio](https://ml.azure.com).

> **Catatan**: Jika pesan yang menunjukkan bahwa non-numerik tidak dapat divisualisasikan ditampilkan, Anda dapat mengabaikannya.

## Menentukan eksekusi dengan performa terbaik

Ketika semua proses telah selesai, Anda dapat menemukan yang terbaik berdasarkan metrik performa yang Anda tentukan (dalam hal ini, yang memiliki AUC terbaik).

In [ ]:
# Print all child runs, sorted by the primary metric
for child_run in run.get_children_sorted_by_primary_metric():
    print(child_run)

# Get the best run, and its metrics and arguments
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
script_arguments = best_run.get_details() ['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print(' -AUC:', best_run_metrics['AUC'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Arguments:',script_arguments)

Sekarang setelah Anda menemukan eksekusi terbaik, Anda dapat mendaftarkan model yang dilatihnya.

In [ ]:
from azureml.core import Model

# Register model
best_run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                        tags={'Training context':'Hyperdrive'},
                        properties={'AUC': best_run_metrics['AUC'], 'Accuracy': best_run_metrics['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **Informasi Selengkapnya**: Untuk informasi selengkapnya tentang Hyperdrive, lihat [dokumentasi Azure ML](https://docs.microsoft.com/azure/machine-learning/how-to-tune-hyperparameters).